In [39]:
from __future__ import print_function
import sys
import cv2
import os
import math
import glob
import hashlib
import time
import numpy as np
import matplotlib.pyplot as plt
import shutil
import site
from PIL import Image
from tqdm.notebook import tqdm
from datetime import datetime
from multiprocessing import Pool
from moviepy.editor import *

%matplotlib inline


In [40]:
#print('numpy:', np.__version__)
#print('pandas:', pd.__version__)
#print('matplotlib:', mplv)
print('OpenCV:', cv2.__version__)
print(list(map(lambda x: x.replace("\\", "/"), site.getsitepackages())))

OpenCV: 4.4.0
['C:/Users/eiuru/Anaconda3/envs/py36-keras-cntk', 'C:/Users/eiuru/Anaconda3/envs/py36-keras-cntk/lib/site-packages']


In [41]:
THRESHOLD = 50.0

In [42]:
def list_files(dir):
    return [os.path.join(dir, p.name) for p in os.scandir(dir)]

def get_filename_without_ext(p):
    return os.path.splitext(os.path.basename(p))[0]

def get_filename_andext(p):
    return os.path.splitext(os.path.basename(p))

In [43]:
def clip_movie(movie_file_path, clips, dst):
    video = VideoFileClip(movie_file_path)
    print(movie_file_path, clips, dst)
    clipsArray = [] 
    for clip in clips:
        [startTime, endTime] = list(map(int, clip.split('-')))
        print(startTime, endTime)
        clip = video.subclip(startTime / video.fps, endTime / video.fps)
        clipsArray.append(clip)
    final = concatenate_videoclips(clipsArray)
    final.write_videofile(dst, fps=video.fps, codec='libx264', audio_codec="aac")
    

In [45]:
def main():
    cwd = os.getcwd()
    movie_dir = os.path.join(cwd, 'movies')
    
    for series_name in ['s1', 's2']:
        movies = list_files(os.path.join(movie_dir, series_name))
        record_dir = os.path.join(cwd, "records", series_name)
        records = list_files(record_dir)
        
        crop_dir = os.path.join(cwd, 'crop_movies', series_name)
        if not os.path.exists(crop_dir):
            os.makedirs(crop_dir, exist_ok=True)

        tmp_dir = os.path.join(cwd, 'crop_tmp', series_name)
        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir, exist_ok=True)

        pbar = tqdm(records)
        for p in pbar:
            with open(p, encoding='UTF-8') as f:
                lines = f.readlines()
                movie_file_path = lines[0].strip()
                clips = list(map(lambda x: x.strip(), lines[1:]))
                if len(clips) == 0: 
                    pbar.update(1)
                    continue
                [filename, ext] = get_filename_andext(movie_file_path)
                hs = hashlib.md5(filename.encode()).hexdigest()
                ascii_filename = "{}{}".format(hs, ext)
                dst = os.path.join(crop_dir, ascii_filename)
                if os.path.exists(dst): 
                    pbar.update(1)
                    continue
                tmp_file_path = os.path.join(tmp_dir, ascii_filename)
                shutil.copy2(movie_file_path, tmp_file_path)
                clip_movie(tmp_file_path, clips, dst)

if __name__ == "__main__":
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    main()
    

2020-11-03 00:14:07
